In [ ]:
!pip install datasets
!pip install rouge_score
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=da6a3aa567b68d5e4a722095c

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

In [ ]:
# from datasets import load_dataset

# data_files = {"train":"/content/drive/MyDrive/Colab Notebooks/dataset.csv"}
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset.csv",encoding='latin-1')

In [ ]:
from datasets import load_dataset

data_files = {"train":"/content/drive/MyDrive/Colab Notebooks/dataset.csv"}

In [ ]:
new_dataset = load_dataset("csv",data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
new_sample = new_dataset["train"].shuffle(seed=42).select(range(1000))
new_sample[:3]

{'Artical Id': [1920, 2370, 81],
 'Summary': ['Ireland set 155 run target for O',
  'Sri Lanka Kulasekara quits tests focus ODIs T20',
  "['PV Sindhu advances after Lakshya Sen"],
 'Content': ['DHARAMSALA: Ireland set a competitive target of 155 for Oman in the fourth match of the World Twenty20, first round group A, here at the Himachal Pradesh Cricket Association Stadium on Wednesday night.After winning the toss and electing to bat first, Ireland made a good start and their three top-order batsmen took them to 112 in 15 overs but later they failed to maintain the tempo and were restricted to 154 for five.Openers captain William Porterfield and Paul Stirling scored 29 each while Gary Wilson top-scored with 38.Oman seamer Munis Ansari captured three wickets for 37.Ireland picked right-arm quick Craig Young in their three-pronged pace attack, and off-spinner Andy McBrine. They left out left-arm spinner George Dockrell and Andy Balbirnie.Â\xa0',
  'strong>NEW DELHI: Sri Lankan pace bowle

In [ ]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['Artical Id', 'Summary', 'Content'],
        num_rows: 3222
    })
})

In [ ]:
new_dataset_clean = new_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
# new_dataset_clean["test"] = new_dataset_clean.pop("test")
nd1= new_dataset_clean["train"].train_test_split(train_size=0.7,seed=42)
nd1["validation"] = nd1.pop("test")
nd1["test"] = new_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
# new_dataset_clean["test"] = new_dataset["test"]
nd1

DatasetDict({
    train: Dataset({
        features: ['Artical Id', 'Summary', 'Content'],
        num_rows: 1803
    })
    validation: Dataset({
        features: ['Artical Id', 'Summary', 'Content'],
        num_rows: 774
    })
    test: Dataset({
        features: ['Artical Id', 'Summary', 'Content'],
        num_rows: 645
    })
})

In [ ]:
ds = nd1.filter(lambda x : len(x["Summary"].split()) > 3)
ds

Filter:   0%|          | 0/1803 [00:00<?, ? examples/s]

Filter:   0%|          | 0/774 [00:00<?, ? examples/s]

Filter:   0%|          | 0/645 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Artical Id', 'Summary', 'Content'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['Artical Id', 'Summary', 'Content'],
        num_rows: 770
    })
    test: Dataset({
        features: ['Artical Id', 'Summary', 'Content'],
        num_rows: 639
    })
})

In [ ]:
from transformers import AutoTokenizer
# model_checkpoint = "google/mt5-small"
model_checkpoint = "google/pegasus-multi_news"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer("I love Hungger games !")
inputs

{'input_ids': [125, 298, 41630, 6867, 727, 110, 147, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁I', '▁love', '▁Hung', 'ger', '▁games', '▁', '!', '</s>']

In [ ]:
max_input_length = 256
max_target_length = 30

def preprocess_function(example):
  model_inputs = tokenizer(example["Content"], max_length=max_input_length, truncation = True)
  labels = tokenizer(example["Summary"], max_length=max_target_length, truncation = True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_ds = ds.map(preprocess_function, batched = True)

Map:   0%|          | 0/1795 [00:00<?, ? examples/s]

Map:   0%|          | 0/770 [00:00<?, ? examples/s]

Map:   0%|          | 0/639 [00:00<?, ? examples/s]

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")

In [ ]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize

def three_sent(text):
  return "\n".join(sent_tokenize(text)[:3])

print(three_sent(ds["train"][1]["Content"]))

Lima, Nov 19, 2016 (AFP) -Chinese President Xi Jinping urged Asia-Pacific leaders Saturday to get on board with Beijing-backed free trade agreements, after Donald TrumpÂ´s election win spelled the likely demise of a US-backed deal.TrumpÂ´s shock victory has cast uncertainty on the future of WashingtonÂ´s key trade initiative in the Pacific Rim, the arduously negotiated, 12-country Trans-Pacific Partnership, or TPP.The brash billionaire campaigned against the accord, which has not yet been ratified in Congress, as a "terrible deal" that would "rape" the United States by sending American jobs to countries with cheaper labor.In a Pacific region hungry for trade, that has left even longtime US allies like Australia and Japan looking to China to fill the void.Beijing, which was excluded from TPP, is pushing two alternatives: the 21-member Free Trade Area of the Asia-Pacific (FTAAP) and a 16-member Regional Comprehensive Economic Partnership (RCEP), which notably includes India but not the U

In [ ]:
def evaluate_baseline(ds, metrics):
  summaries = [three_sent(text) for text in ds["Content"] if text is not None] #filter out the None values in the Content
  references = [text for text in ds["Summary"] if text is not None] #filter out the None values in the Summary
  #Check if both lists are empty
  if not summaries or not references:
    print("Warning: either summaries or references list is empty, returning empty dictionary")
    return {} # return an empty dictionary if either list is empty
  return metrics.compute(predictions=summaries, references=references) #pass references instead of reference

In [ ]:
import pandas as pd

score = evaluate_baseline(ds["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 11.54, 'rouge2': 5.82, 'rougeL': 9.79, 'rougeLsum': 10.09}

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-multi_news and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size=4
num_train_epoch = 3
logging_step = len(tokenized_ds['train'])
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir = "pegasus-finetuned-dataset",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 1,
    num_train_epochs = num_train_epoch,
    predict_with_generate = True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import numpy as np

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens= True)
  decoded_preds=["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
  result = rouge_score.compute(
      predictions = decoded_preds, references= decoded_labels, use_stemmer=True
  )
  result = {key: value*100 for key, value in result.items()}
  return {k: round(v, 4) for k, v in result.items()}


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:

tokenized_ds = tokenized_ds.remove_columns(
    ds["train"].column_names
)

ValueError: Column name ['Summary', 'Artical Id', 'Content'] not in the dataset. Current columns in the dataset: ['input_ids', 'attention_mask', 'labels']

In [ ]:
features = [tokenized_ds["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[ 1034, 92473, 51586,  3220,   143,  1880,  3958,   158, 58387,   420,
         43830, 51586,  3220,   148,  4470,   169,  1222,  2284,   115,  2269,
         15643,   134,   698,  8863,   244,  2868,   295,   134,   109,  3165,
          9554,   107, 51586,  3220,  3999,   178,   148,   126,   115,   342,
           111,   117,   806,   505,   112,  1023, 15643,   134,   349,  1099,
           107,   159,  3937,  1121,  3921,   112,  7043,   136, 33231,   813,
           121, 52424,   112,  1433,  1225, 12075,   115,   533,   702,   130,
           178,   239,   978,   782,   112,   109, 35582,  1223,   894,  3164,
          7608, 34074,   122,  9833,  6796,  3574,   112,   795,   169, 14806,
           190,  1980,   107,   159,  2128, 25178,   127,   323,   112,  4681,
           134,   109, 35582,  1223,   894,  3164,  7608, 34074,  1215,  1842,
          1350,   738,   115,   351,  5321,   107,   139,   702,   138,   129,
           886,   134,   109,   982,  

In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"],
    data_collator = data_collator,
    tokenizer= tokenizer,
    compute_metrics= compute_metrics
)

<ipython-input-33-efe72842a348>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,2.381658,27.215100,12.546700,23.853400,24.148900
2,1.978000,2.269829,28.826300,13.648500,25.458600,25.765300
3,1.936400,2.270538,29.026900,13.952400,25.669500,25.979300


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=1347, training_loss=1.9213017588999861, metrics={'train_runtime': 2475.6576, 'train_samples_per_second': 2.175, 'train_steps_per_second': 0.544, 'total_flos': 3884015808184320.0, 'train_loss': 1.9213017588999861, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'eval_loss': 2.270538091659546,
 'eval_rouge1': 29.0269,
 'eval_rouge2': 13.9524,
 'eval_rougeL': 25.6695,
 'eval_rougeLsum': 25.9793,
 'eval_runtime': 420.3303,
 'eval_samples_per_second': 1.832,
 'eval_steps_per_second': 0.459,
 'epoch': 3.0}

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Finetuned-model")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Finetuned-model-tokenizer")

('/content/drive/MyDrive/Colab Notebooks/Finetuned-model-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Finetuned-model-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Finetuned-model-tokenizer/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/Finetuned-model-tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Finetuned-model-tokenizer/tokenizer.json')

In [ ]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Finetuned-model")

In [ ]:
finetuned_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Finetuned-model-tokenizer")

In [ ]:
from transformers import pipeline

model_id = finetuned_model
tokenizer = finetuned_tokenizer # Evaluate the f-string correctly
summarizer = pipeline("summarization", model=model_id,tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
def print_summary(idx):
    Content = ds["test"][idx]["Content"]
    Exist_summary = ds["test"][idx]["Summary"]
    summary = summarizer(ds["test"][idx]["Content"])[0]["summary_text"]
    print(f"'>>> Content: {Content}'")
    print(f"\n'>>> Exist_summary: {Exist_summary}'")
    print(f"\n'>>> Summary: {summary}'")

In [ ]:
print_summary(1)

Your max_length is set to 256, but your input_length is only 210. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=105)


'>>> Content: strong>NEW DELHI: Sri Lankan pace bowler Nuwan Kulasekara quit test cricket on Wednesday in order to focus his attention on the sport's shorter formats, in which he is considered a greater threat with the ball.</strong"I would like to retire from playing test cricket with immediate effect," the 33-year-old said in a letter to Sri Lanka Cricket (SLC)."After much thought, I believe it is the right time to retire and I hope it will allow me to better prepare for ODI and T20 cricket, which I would like to continue playing for Sri Lanka subject to good performance and selection."Despite lacking express pace, Kulasekara made the most of his ability to swing the new ball to claim 48 test wickets since his 2005 debut against New Zealand in Napier.Kulasekara played the last of his 21 tests against England at Lord's two years ago but has been a standout performer in one-dayers, with 186 wickets from 173 matches, and was ranked the top ODI bowler in March 2009.'

'>>> Exist_summary: